# 1.for_all_pos 

```bash
bsub -M 2000 -e /nfs/research/goldman/zihao/compViridian_2/3_combination/2_pg_merge_all_pos_errorChecking_error.txt 'python3 /nfs/research/goldman/zihao/compViridian_2/3_combination/2_pg_merge_all_pos.py'
```

In [ ]:
import pandas as pd
import glob
import os

files = glob.glob('/nfs/research/goldman/zihao/Datas/p2_comp_viridian/3_combination/*.txt')

for file in files:
    try:
        basename = os.path.basename(file)
        id = basename[:-4]

        df = pd.read_csv(file, sep='\t')

        df_cov_path = f"/nfs/research/goldman/zihao/Datas/p1/File_5_coverage/Decompress/{id}_coverage.txt"
        if not os.path.exists(df_cov_path):
            continue
        df_cov = pd.read_csv(df_cov_path, sep='\t')

        df_ann_path = f"/nfs/research/goldman/zihao/Datas/p1/File_5_annot/Decompress/{id}_annot.txt"
        if not os.path.exists(df_ann_path):
            continue
        df_ann = pd.read_csv(df_ann_path, sep='\t')

        df_merge = pd.concat([df, df_cov['RATIO'], df_ann['AF'], df_ann['SB']], axis=1)

        df_merge.to_csv(f"/nfs/research/goldman/zihao/Datas/p2_comp_viridian/3.1_merge_sb_af/{id}.txt", index=False, sep='\t')

    except Exception as e:
        print(f"An error occurred while processing {file}: {e}")

#### 2.for_err_pos 

```bash
bsub -M 2000 -e /nfs/research/goldman/zihao/errorsProject_1/Part1_2_for_assemble/Part2_Compare/3_for_err_pos_errorChecking_error.txt 'python3 /nfs/research/goldman/zihao/errorsProject_1/Part1_2_for_assemble/Part2_Compare/3_for_err_pos.py'
```

In [ ]:
import os
import pandas as pd

def read_file(file_path, delimiter='\t'):
    """Reads a file into a pandas DataFrame."""
    return pd.read_csv(file_path, delimiter=delimiter)

def fetch_file_path(folder_path, file_id, extension='.txt'):
    """Builds the file path from the folder path and file id."""
    return os.path.join(folder_path, f'{file_id}{extension}')

def match_row(df, column, value):
    """Finds rows in a DataFrame that match a specific value in a specific column."""
    return df[df[column] == value]

def append_columns(df_source, df_target, index, columns):
    """Appends columns from a source DataFrame to a target DataFrame."""
    df_target.loc[index, columns] = df_source[columns].values.tolist()[0]

def save_file(df, file_path, columns, delimiter='\t', index=False):
    """Saves a DataFrame to a file."""
    df[columns].to_csv(file_path, sep=delimiter, index=index)

# Specify paths and column names 
a_file_path = '/nfs/research/goldman/zihao/errorsProject_1/MAPLE/new_version_MAY/output_modified.txt'
b_folder_path = "TEST/"
b_column_names = ['position', 'nucleotide_martin', 'nucleotide_origin', 'label', 'label2', 'label_mar', 'label_ori']
output_file_path = 'output_data.txt'

# Load the data from the 'A' file
a_data = read_file(a_file_path)

# Iterate through each row of the 'A' data
for index, row in a_data.iterrows():
    # Fetch the path of the corresponding 'B' file
    b_file_path = fetch_file_path(b_folder_path, row['ID'])
    
    # Continue to the next iteration if the 'B' file doesn't exist
    if not os.path.isfile(b_file_path):
        continue

    # Load the data from the 'B' file
    b_data = read_file(b_file_path)

    # Find the matching row in the 'B' data
    matched_row = match_row(b_data, 'position', row['Position'])

    # Continue to the next iteration if no matching row was found
    if matched_row.empty:
        print(f'No matching row found in {b_file_path} for ID {row["ID"]} and position {row["Position"]}.')
        continue

    # Append the necessary columns to the 'A' data
    append_columns(matched_row, a_data, index, b_column_names)

# Save the modified 'A' data
save_file(a_data, output_file_path, ['ID', 'Position'] + b_column_names)

print(f"Data saved to {output_file_path}")

In [9]:
df_merge

position nucleotide_martin nucleotide_origin  label_masked  label_mar  \
0             1                 -                 n             1          1   
1             2                 -                 n             1          1   
2             3                 -                 n             1          1   
3             4                 -                 n             1          1   
4             5                 -                 n             1          1   
...         ...               ...               ...           ...        ...   
29898     29899                 -                 n             1          1   
29899     29900                 -                 n             1          1   
29900     29901                 -                 n             1          1   
29901     29902                 -                 n             1          1   
29902     29903                 -                 -             1          1   

       label_ori  label_same  RATIO   AF  SB  
0              1           2    0.0  0.0   0  
1              1           2    0.0  0.0   0  
2              1           2    0.0  0.0   0  
3              1           2    0.0  0.0   0  
4              1           2    0.0  0.0   0  
...          ...         ...    ...  ...  ..  
29898          1           2    0.0  0.0   0  
29899          1           2    0.0  0.0   0  
29900          1           2    0.0  0.0   0  
29901          1           2    0.0  0.0   0  
29902          1           2    0.0  0.0   0  

[29903 rows x 10 columns]